In [143]:
#Import library
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [216]:
#Read data
train=pd.read_csv('../input/netflix-appetency/train.csv')
test=pd.read_csv('../input/netflix-appetency/test.csv')
sample=pd.read_csv('../input/netflix-appetency/sample_submission.csv')

In [217]:
train.head()

,id,target,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_497,feature_498,feature_499,feature_500,feature_501,feature_502,feature_503,feature_504,feature_505,feature_506
0,0,0,C0,C0,C1,C5,C11,37.56,54.756667,54.756667,...,0,0,0,0,0,0,0,0,0,0
1,1,0,C0,C0,C3,C5,C1,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2,0,C0,C0,C3,C5,C2,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,3,0,C0,C0,C1,C5,C1,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,4,1,C0,C0,C3,C3,C11,37.48,37.480000,37.161333,...,0,0,0,0,0,0,0,0,0,0


In [218]:
#create train, target, and test
target=train["target"]
train=train.drop(["id","target"],axis=1)
test=test.drop(["id"],axis=1)

In [219]:
train.shape, test.shape

((70000, 507), (30000, 507))

In [220]:
#Get columns with more than 50% missing values
Missing_features=train.loc[:,(train.isnull().sum()/train.shape[0])>0.5]
#Drop them from train and test 
train=train.drop(Missing_features.columns,axis=1)
test=test.drop(Missing_features.columns,axis=1)

In [221]:
#Categorical features name
Categorical_Features=train.select_dtypes(include=['object']).columns
        
#Numeric features name
Numeric_Features=train.select_dtypes(exclude=['object']).columns[2:] #skip id,target

In [222]:
#Fillna numeric features  with mean
train[Numeric_Features]=train[Numeric_Features].fillna(train[Numeric_Features].mean())
test[Numeric_Features]=test[Numeric_Features].fillna(test[Numeric_Features].mean())

In [223]:
#check NA in train
print(train.loc[:,(train.isnull().sum())>0].shape)
#check NA in test
print(test.loc[:,(test.isnull().sum())>0].shape)

(70000, 7)
(30000, 7)


In [224]:
datetime_Features=train.loc[:,(train.isnull().sum()/train.shape[0])>0]
#Drop from train/test
train.drop(datetime_Features.columns,axis=1,inplace=True)
test.drop(datetime_Features.columns,axis=1,inplace=True)

In [226]:
#Drop column contains one unique value
one_unique=train.loc[:,train.nunique()==1]
train.drop(one_unique.columns,axis=1,inplace=True)
test.drop(one_unique.columns,axis=1,inplace=True)

In [227]:
train.shape, test.shape

((70000, 419), (30000, 419))

In [228]:
#rescale train and test
scaler = StandardScaler() 
scaler.fit(Xtrain) 
Xtrain = scaler.transform(Xtrain)
Xtest = scaler.transform(Xtest)

In [229]:
#neural network from sklearn for embeddings
mlp = MLPClassifier(solver='adam', random_state=1,max_iter=2000, warm_start=True)
#clf.fit(Xtrain, y)



In [232]:
parameter_space = {
    'hidden_layer_sizes': [(12,),(10,), (8,), (6,),(4,),(2,)],
    'activation': ['logistic', 'relu'],
    'alpha': [0.05,0.01,0.5],
}

In [233]:
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(Xtrain, y)

GridSearchCV(cv=3,
             estimator=MLPClassifier(max_iter=2000, random_state=1,
                                     warm_start=True),
             n_jobs=-1,
             param_grid={'activation': ['logistic', 'relu'],
                         'alpha': [0.05, 0.01, 0.5],
                         'hidden_layer_sizes': [(12,), (10,), (8,), (6,), (4,),
                                                (2,)]})

In [127]:
print (clf.cv_results_['params'])
print(clf.cv_results_['mean_test_score'])

[{'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20, 10), 'learning_rate': 'constant'}, {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20, 10), 'learning_rate': 'adaptive'}, {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20,), 'learning_rate': 'constant'}, {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20,), 'learning_rate': 'adaptive'}, {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (10,), 'learning_rate': 'constant'}, {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (10,), 'learning_rate': 'adaptive'}, {'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (20, 10), 'learning_rate': 'constant'}, {'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (20, 10), 'learning_rate': 'adaptive'}, {'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (20,), 'learning_rate': 'constant'}, {'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (20,), 'learning_rate': 'adaptive'}, {'act

In [142]:
clf.cv_results_['params'][16]

{'activation': 'relu',
 'alpha': 0.0001,
 'hidden_layer_sizes': (10,),
 'learning_rate': 'constant'}

In [120]:
sklearn_nn_predict=clf.predict_proba(Xtest)
#Read the sample_submission
sample['target']=sklearn_nn_predict[:,1]
sample.to_csv('./sklearn_nn.csv',index=False)
sample.head(10)

,id,target
0,5,0.514764
1,7,0.239698
2,9,0.219323
3,11,0.117124
4,14,0.225515
5,15,0.326341
6,19,0.127003
7,21,0.100690
8,25,0.261394
9,29,0.194130
